## *Jeu de données IRVE*
-------------------
# Analyse des relations entre champs (intégrité des données)
------------------------------
## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- les principaux résultats sont regroupés sur un [tableau de bord](https://nbviewer.org/github/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/Analyse/IRVE_indicateurs.ipynb) 
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par l'identification d'incohérences de relations
- l'identification des incohérences permet de trouver des stratégies de réduction des écarts (dans l'exemple ci-dessous, on passe 36% d'écart à 3,8 %)  
- l'analyse des données permet de (re)construire le modèle de données qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 5 % des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Séparation doublons station - date de maj](#6---Séparation-doublons-station---date-de-maj)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d'erreurs résiduelles](#8---Exemples-d\'erreurs-résiduelles)

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve/Analyse)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-2024xxxx.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [1]:
from base64 import b64encode
from IPython.display import Image, display
with open('IRVE_modele.txt', 'r', encoding="utf-8") as f:
    modele = f.read()
display(Image(url="https://mermaid.ink/img/" + b64encode(modele.encode("ascii")).decode("ascii")))

------
# 2 - Initialisation
## initialisation logicielle

In [1]:
from datetime import datetime
import json
from tab_dataset.cdataset import Cdataset
import pandas as pd
import ntv_pandas as npd
import pathlib

def analyse_integrite(data, fields, affiche=True, indic=True):
    '''analyse les relations du DataFrame 'data' définies dans le dict 'fields'.
    Le nombre de ligne en erreur par relation (dict) est retourné et optionnellement ('indic') affiché. 
    La liste des lignes en erreur est optionnellement ajoutée ('affiche') à 'data' sous forme de champs booléens par relation.
    '''
    dic_errors = Cdataset(data).check_relationship(fields)
    dic_count = {name: len(errors) for name, errors in dic_errors.items()}
    if affiche:
        for name, total in dic_count.items():
            print('{:<50} {:>5}'.format(name, total))
    if indic:
        data['ok'] = True
        for name, errors in dic_errors.items():
            data[name] = True
            data.loc[errors, name] = False
            data['ok'] = data['ok'] & data[name] 
    return dic_count

def add_nbre_pdc(data, affiche=True):
    '''ajoute un champ avec le nombre de pdc calculés et le compare au champ 'nbre_pdc' '''
    data['nb_pdc_calc'] = data.groupby('id_station_itinerance')['index'].transform('count')
    data['nbre_pdc_ok'] = data['nb_pdc_calc'] == data['nbre_pdc']    
    data['calc-nbre'] = data['nb_pdc_calc'] - data['nbre_pdc']
    if affiche:
        ecart_nbre = len(data) - data['nbre_pdc_ok'].sum()
        print('{:<20} {:>5}'.format('nbre_pdc_ko', ecart_nbre))
        print(round(ecart_nbre / len(data) * 100), ' %')

## initialisation des données
- lecture du fichier issu de l'api

In [2]:
dates = ['2023-03-03', '2023-04-17', '2023-05-02', '2023-05-24', '2023-06-17', '2023-07-04', '2023-07-23', '2023-08-08',
         '2023-09-06', '2023-09-19', '2023-10-04', '2023-10-21', '2023-10-31', '2023-11-23', '2023-12-14', '2023-12-29',
         '2024-01-15', '2024-02-07', '2024-02-28']
date= dates[-1]
log = {'date_irve': date, 
    'file': 'consolidation-etalab-schema-irve-statique-v-2.3.1-'+date[:4]+date[5:7]+date[8:]+'.csv', # après 2024-02-06
    #'file': 'consolidation-etalab-schema-irve-statique-v-2.2.0-'+date[:4]+date[5:7]+date[8:]+'.csv',
    'chemin': str(pathlib.Path(npd.__file__).parent.parent.parent/"Environmental-Sensing"/"python"/"Validation"/"irve"/"Analyse")
    # 'chemin': 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/gen-ntv/python/Validation/irve/Analyse/'}
    # log = {'date_irve': '2022-06-06', 'file': 'consolidation-etalab-schema-irve-v-2.0.2-20220606-propre2.csv',
    #'chemin': 'D:\\philippe\\python ESstandard\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'
    #'chemin': 'C:\\Users\\phili\\github\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'
      }
irve = pd.read_csv(log['chemin'] + '/' + log['file'], sep=',', low_memory=False)
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 

nombre de lignes :  87974


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [3]:
# complément à inclure dans le schéma de données
fields = [
 # relation unicité des pdl
     { "name": "index",
       "relationship" : { "parent" : "id_pdc_itinerance", "link" : "coupled" }},   
 # relations inter entités
 { "name": "contact_operateur",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nom_enseigne",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "coordonneesXY",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "id_station_itinerance",
   "relationship" : { "parent" : "id_pdc_itinerance",     "link" : "derived" }},
 # relations intra entité - station
 { "name": "nom_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "implantation_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 #{ "name": "date_maj",
 #  "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nbre_pdc",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "condition_acces",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "horaires",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "station_deux_roues",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 # relations intra entité - localisation
 { "name": "adresse_station",
   "relationship" : { "parent" : "coordonneesXY",         "link" : "derived" }} ]

# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory = ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
             'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
             'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
             'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
             'date_maj', 'last_modified']
controles = ['index - id_pdc_itinerance', 'contact_operateur - id_station_itinerance', 'nom_enseigne - id_station_itinerance',
             'coordonneesXY - id_station_itinerance', 'id_station_itinerance - id_pdc_itinerance',
             'nom_station - id_station_itinerance', 'implantation_station - id_station_itinerance',
             'nbre_pdc - id_station_itinerance', 'condition_acces - id_station_itinerance', 'horaires - id_station_itinerance',
             'station_deux_roues - id_station_itinerance', 'adresse_station - coordonneesXY']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- de l'ordre de 4 % des points de charge sont hors itinerance

In [4]:
# séparation des données
data = irve
data[['id_station_itinerance','id_pdc_itinerance']] = data[['id_station_itinerance','id_pdc_itinerance']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'], 'soit : ', len(non_concerne)/(len(non_concerne)+len(itinerance)))
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  881 soit :  0.010014322413440335
nombre de pdc en itinerance   :  87093


In [5]:
# arborescence des champs et nombre de valeurs différentes
print(itinerance_init.npd.analysis().tree())

-1: root-derived (87093)
   0 : index (0 - 87093)
   1 : contact_operateur (86926 - 167)
   2 : nom_enseigne (82990 - 4103)
   3 : coordonneesXY (62549 - 24544)
   4 : adresse_station (63064 - 24029)
   5 : id_station_itineranc (54084 - 33009)
   6 : nom_station (62269 - 24824)
   7 : implantation_station (87088 - 5)
   8 : nbre_pdc (87031 - 62)
   9 : condition_acces (87091 - 2)
   10: horaires (86499 - 594)
   11: station_deux_roues (87087 - 6)
   12: id_pdc_itinerance (13401 - 73692)
   13: date_maj (86347 - 746)
   14: last_modified (86614 - 479)


-----------------------
## 4 - Bilan initial intégrité
- 34 % des lignes présentent un défaut d'intégrité

In [6]:
# séparation données bonnes (itinerance_ok_1) et données résiduelles (itinerance_1)
res = analyse_integrite(itinerance_init, fields)
itinerance_ok_1 = itinerance_init.loc[itinerance_init.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance_init.loc[~itinerance_init.ok, relations].reset_index(drop=True)
itinerance_init = itinerance_init.loc[:, relations]
log['init_ok'] = len(itinerance_ok_1)
log['init_ko'] = len(itinerance_1)
print("\nnombre d'enregistrements sans erreurs : ", log['init_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['init_ko'])
print("taux d'erreur : ", round(log['init_ko'] / log['pdc_en_itinerance'] * 100), ' %')

index - id_pdc_itinerance                          26242
contact_operateur - id_station_itinerance          13557
nom_enseigne - id_station_itinerance               14170
coordonneesXY - id_station_itinerance              14486
id_station_itinerance - id_pdc_itinerance          12962
nom_station - id_station_itinerance                13515
implantation_station - id_station_itinerance        6320
nbre_pdc - id_station_itinerance                    5532
condition_acces - id_station_itinerance               60
horaires - id_station_itinerance                      90
station_deux_roues - id_station_itinerance         13339
adresse_station - coordonneesXY                     1976

nombre d'enregistrements sans erreurs :  57708
nombre d'enregistrements avec au moins une erreur :  29385
taux d'erreur :  34  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- 35% des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de réduire de 50% le nombre de lignes erronnées

In [7]:
# séparation doublons pdc (doublons_pdc) et données résiduelles (itinerance_2)
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by=['date_maj', 'last_modified']).duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
#doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
log['doublons_pdc'] = len(doublons_pdc)
log['sans_doublons_pdc'] = len(itinerance_2)
print('nombre de doublons pdc : ', log['doublons_pdc'], ' soit ', round(log['doublons_pdc']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_pdc'], ' soit ', round(log['sans_doublons_pdc']/log['init_ko'] * 100), ' %')

nombre de doublons pdc :  13401  soit  46  %
nombre de pdc sans doublon   :  15984  soit  54  %


In [8]:
# séparation données bonnes (itinerance_ok_3) et données résiduelles (itinerance_3)
res = analyse_integrite(itinerance_2, fields)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
log['etape3_ok'] = len(itinerance_ok_3)
log['etape3_ko'] = len(itinerance_3)
print("\nnombre d'enregistrements sans erreurs : ", log['etape3_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape3_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance            796
nom_enseigne - id_station_itinerance                 799
coordonneesXY - id_station_itinerance               1859
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                 1479
implantation_station - id_station_itinerance         647
nbre_pdc - id_station_itinerance                     955
condition_acces - id_station_itinerance               28
horaires - id_station_itinerance                      58
station_deux_roues - id_station_itinerance          1357
adresse_station - coordonneesXY                     1426

nombre d'enregistrements sans erreurs :  11934
nombre d'enregistrements avec au moins une erreur :  4050


-----------------------
## 6 - Séparation doublons station - date de maj
- la suppression des anciens pdc permet de réduire de 25% supplémentaire le nombre de lignes erronnées
- les dernières erreurs correspondent à des stations associées à 43 opérateurs et sont liées à des causes multiples

In [9]:
# séparation doublons stations (doublons_stat_maj) et données résiduelles (itinerance_4)
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
log['doublons_station'] = len(doublons_stat_maj)
log['sans_doublons_station'] = len(itinerance_4)
print('nombre de doublons stations : ', log['doublons_station'], ' soit ', 
      round(log['doublons_station']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_station'])

nombre de doublons stations :  1222  soit  4  %
nombre de pdc sans doublon   :  2828


In [10]:
# séparation données bonnes (itinerance_ok_5) et données résiduelles (itinerance_5 / itinerance_5_full)
res = analyse_integrite(itinerance_4, fields)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
log['etape5_ok'] = len(itinerance_ok_5)
log['etape5_ko'] = len(itinerance_5)
print("\nnombre d'enregistrements sans erreurs : ", log['etape5_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape5_ko'], ' soit ', 
      round(log['etape5_ko']/log['etape3_ko'] * 100), ' %')

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              2
nom_enseigne - id_station_itinerance                   5
coordonneesXY - id_station_itinerance                353
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  386
implantation_station - id_station_itinerance         119
nbre_pdc - id_station_itinerance                     416
condition_acces - id_station_itinerance                4
horaires - id_station_itinerance                      29
station_deux_roues - id_station_itinerance           448
adresse_station - coordonneesXY                     1303

nombre d'enregistrements sans erreurs :  445
nombre d'enregistrements avec au moins une erreur :  2383  soit  59  %


In [11]:
# structure des données présentant une erreur et nombre de valeurs
print(itinerance_5.npd.analysis().tree())

-1: root-derived (2383)
   0 : index (0 - 2383)
   1 : contact_operateur (2336 - 47)
   2 : nom_enseigne (2210 - 173)
   3 : coordonneesXY (1971 - 412)
   4 : adresse_station (1795 - 588)
   5 : id_station_itineranc (1243 - 1140)
      13: date_maj (1040 - 100)
   6 : nom_station (1854 - 529)
   7 : implantation_station (2378 - 5)
   8 : nbre_pdc (2359 - 24)
   9 : condition_acces (2381 - 2)
   10: horaires (2333 - 50)
   11: station_deux_roues (2378 - 5)
   12: id_pdc_itinerance (0 - 2383)
   14: last_modified (2311 - 72)


In [12]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  4  erreurs :  8


-----------------------
## 7 - Synthèse


### fichiers
Génération des fichiers intégrant les défauts d'intégrité :
- fichier csv des lignes résiduelles à traiter (IRVE_itinerance_residuel)
- fichier csv des données itinerance avec indicateur des données à corriger ou à ignorer (IRVE_itinerance_complet)
- fichier csv des données itinerance valides (IRVE_itinerance_valide)
- fichier csv des doublons (IRVE_itinerance_doublons)

In [13]:
# consolidation des données
itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
itinerance_doublons = itinerance[itinerance['doublons_a_supprimer']].reset_index(drop=True)
print('total des doublons à supprimer : ', len(itinerance_doublons))
itinerance_ok = itinerance[~itinerance['lignes_ko']].reset_index(drop=True)
print('nombre de pdc avec controles ok : ', len(itinerance_ok))

total des lignes à corriger :  2383
total des doublons à supprimer :  14623
nombre de pdc avec controles ok :  70087


In [14]:
#génération des fichiers
extension = log['date_irve'] +'.csv'
itinerance_5_full.to_csv('IRVE_itinerance_residuel' + extension)
itinerance.to_csv('IRVE_itinerance_complet' + extension)
itinerance_ok.to_csv('IRVE_itinerance_valide' + extension)
itinerance_doublons.to_csv('IRVE_itinerance_doublons' + extension)
log['IRVE_itinerance_residuel' + extension] = len(itinerance_5_full)
log['IRVE_itinerance_complet' + extension] = len(itinerance)
log['IRVE_itinerance_valide' + extension] = len(itinerance_ok)
log['IRVE_itinerance_valide_stat' + extension] = len(itinerance_ok.drop_duplicates('id_station_itinerance', keep='last'))
log['IRVE_itinerance_doublons' + extension] = len(itinerance_doublons)

### vérification de l'intégrité

In [15]:
# vérification de l'absence d'erreurs
res = analyse_integrite(itinerance_ok.loc[:, relations], fields, affiche=False)
log['bilan_erreurs'] = sum(res.values())
log['date'] = datetime.now().isoformat()
print('bilan intégrité :')
print('    erreurs : ', log['bilan_erreurs'])

bilan intégrité :
    erreurs :  0


In [16]:
# structure des données bonnes
#print(Cdataset(itinerance_ok.loc[:, mandatory]).tree())
print(itinerance_ok.loc[:, mandatory].npd.analysis().tree())

-1: root-derived (70087)
   4 : id_station_itineranc (39836 - 30251)
      1 : nom_enseigne (26301 - 3950)
      2 : coordonneesXY (10462 - 19789)
         0 : contact_operateur (19638 - 151)
         3 : adresse_station (546 - 19243)
      5 : nom_station (10284 - 19967)
      6 : implantation_station (30246 - 5)
      7 : nbre_pdc (30189 - 62)
      8 : condition_acces (30249 - 2)
      9 : horaires (29718 - 533)
      10: station_deux_roues (30245 - 6)
      19: paiement_autre (30242 - 9)
   11: id_pdc_itinerance (0 - 70087)
   12: puissance_nominale (69984 - 103)
   13: prise_type_ef (70079 - 8)
   14: prise_type_2 (70080 - 7)
   15: prise_type_combo_ccs (70080 - 7)
   16: prise_type_chademo (70080 - 7)
   17: prise_type_autre (70080 - 7)
   18: paiement_acte (70079 - 8)
   20: reservation (70080 - 7)
   21: accessibilite_pmr (70083 - 4)
   22: restriction_gabarit (69984 - 103)
   23: date_maj (69490 - 597)
   24: last_modified (69676 - 411)


### Indicateurs

In [17]:
# génération des indicateurs liés aux règles
regles = ['Pdc non unique', 'Station multi-operateurs', 'Station multi-enseignes', 'Station multi-localisations', 
          'Pdc multi-stations', 'station avec plusieurs noms', 'station multi-implantations', 
          'nombre de pdc par station incoherent', 'station multi-acces', 'station multi-horaires', 
          'acces deux-roues incoherent', 'localisation multi-adresses']
principal = [16, 17, 18, 19, 20]
secondaire = [21, 22, 23, 24, 25, 26, 27]
irve = itinerance_5_full # residuel
total = len(irve)
indic = {}

irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']
indic['principal pdc'] = int(total - irve['principal'].sum())
indic['secondaire pdc'] = int(total - irve['secondaire'].sum())

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
indic['principal stat'] = len(irve_p)
indic['secondaire stat'] = len(irve_s)



In [18]:
# ajout des principaux opérateurs contributeur de défauts
operateurs = list(itinerance_5.drop_duplicates('contact_operateur')['contact_operateur'])
erreurs_op = [len(itinerance_5.loc[itinerance_5.contact_operateur == operateur]) for operateur in operateurs]
err_op, oper = tuple(zip(*(sorted(zip(erreurs_op, operateurs), reverse=True))[:5]))
indic['operateurs'] = list(oper)
indic['erreurs_operateurs'] = list(err_op)

print(len(err_op), ' opérateurs représentent : ', round(sum(err_op) / sum(erreurs_op) * 100), ' % des écarts\n')
for err, op in zip(indic['erreurs_operateurs'], indic['operateurs']):
    print(op, err)

5  opérateurs représentent :  66  % des écarts

supervision-ev.france@totalenergies.com 554
info@ionity.eu 423
sav@izivia.com 234
support@alizecharge.fr 190
contact@e55c.com 161


In [19]:
# ajout de l'indicateur d'écart entre nombre de pdc calculés et saisis
add_nbre_pdc(irve, affiche=True)
indic['nb_pdc_calc-nbre_pdc'] = int(irve['calc-nbre'].sum())

nbre_pdc_ko           1268
53  %


In [20]:
# stockage des indicateurs
log |= indic
log_file = log['chemin'] + '/' + 'logfile.txt'
#with open('logfile.txt', 'a', encoding="utf-8") as f:
with open(log_file, 'a', encoding="utf-8") as f:
    f.write(json.dumps(log) + '\n')

-----------------------
## 8 - Exemples d'erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (3) (109 pdc)
- exemple station 'FR55CP92140' avec 29 pdc ! : 5 coordonnées XY, 5 nom_station, 2 nbre_pdc)

In [21]:
maxi.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1951,70036,support@alizecharge.fr,Reveo,"[3.083424, 42.526063]",COLLIOURE - Place Du 8 Mai 1945 B01,FRS66P66053001,COLLIOURE - Place Du 8 Mai 1945 B01,Voirie,2,Accès libre,Mo-Su 00:00-23:59,False,FRS66E660530011,2024-02-27,2024-02-27T11:11:36.071000+00:00
1952,70037,support@alizecharge.fr,Reveo,"[3.083424, 42.526063]",COLLIOURE - Place Du 8 Mai 1945 B01,FRS66P66053001,COLLIOURE - Place Du 8 Mai 1945 B01,Voirie,2,Accès libre,Mo-Su 00:00-23:59,False,FRS66E660530012,2024-02-27,2024-02-27T11:11:36.071000+00:00
1953,70038,support@alizecharge.fr,Reveo,"[3.083424, 42.526063]",Place Du 8 Mai 1945 66190 COLLIOURE,FRS66P66053001,COLLIOURE - Place Du 8 Mai 1945 B02,Station dédiée à la recharge rapide,1,Accès libre,Mo-Su 00:00-23:59,False,FRS66E660530013,2024-02-27,2024-02-27T11:11:36.071000+00:00
2292,80816,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550401,2024-02-27,2024-02-27T11:11:36.071000+00:00
2293,80817,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550402,2024-02-27,2024-02-27T11:11:36.071000+00:00
2294,80818,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550403,2024-02-27,2024-02-27T11:11:36.071000+00:00
2295,80819,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550404,2024-02-27,2024-02-27T11:11:36.071000+00:00
2296,80820,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550405,2024-02-27,2024-02-27T11:11:36.071000+00:00


In [22]:
itinerance_4.loc[itinerance_4.id_station_itinerance == 'FR55CP92140', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
86,529,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553311,2022-11-03,2024-02-05T15:16:49.826000+00:00
87,530,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553321,2022-11-03,2024-02-05T15:16:49.826000+00:00
88,531,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553331,2022-11-03,2024-02-05T15:16:49.826000+00:00
89,532,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553341,2022-11-03,2024-02-05T15:16:49.826000+00:00
90,533,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553351,2022-11-03,2024-02-05T15:16:49.826000+00:00
91,534,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553361,2022-11-03,2024-02-05T15:16:49.826000+00:00
92,535,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.255335, 48.786899]",HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404879898422628411,2022-11-03,2024-02-05T15:16:49.826000+00:00
93,536,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.255335, 48.786899]",HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404879898422628421,2022-11-03,2024-02-05T15:16:49.826000+00:00
94,537,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.255335, 48.786899]",HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404879898422628431,2022-11-03,2024-02-05T15:16:49.826000+00:00
95,538,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.255335, 48.786899]",HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404879898422628441,2022-11-03,2024-02-05T15:16:49.826000+00:00


### Repartition des defauts par operateur
Répartition pour les 5 opérateurs présentant le plus de défauts

In [23]:
print('PdC avec une incohérence : ')
for operateur in oper:
    print('\n', sum(itinerance_4.contact_operateur == operateur))
    print('Répartition des incohérences ', operateur, ' :')
    for cont in controles:
        #print(cont, sum((itinerance_4['contact_operateur'] == 'supervision-ev.france@totalenergies.com') & ~(itinerance_4[cont])))
        print('{:<50} {:>5}'.format(cont, sum((itinerance_4['contact_operateur'] == operateur) & ~(itinerance_4[cont]))))

PdC avec une incohérence : 

 595
Répartition des incohérences  supervision-ev.france@totalenergies.com  :
index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinerance                   0
coordonneesXY - id_station_itinerance                 38
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                   30
implantation_station - id_station_itinerance           0
nbre_pdc - id_station_itinerance                       5
condition_acces - id_station_itinerance                0
horaires - id_station_itinerance                      10
station_deux_roues - id_station_itinerance           448
adresse_station - coordonneesXY                       60

 423
Répartition des incohérences  info@ionity.eu  :
index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinerance     

### Defauts operateur ALIZECHARGE
Les défauts identifiés concernent des stations utilisant un même identifiant. Par exemple, la station 'FRADPP91479018' contient 82 points de recharge avec plusieurs adresses et un nombre de points de rechage variable. Cet exemple concerne des stations implantées sur plusieurs étages d'un parking (on devrait donc a minima identifier une station par étage).

In [24]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'support@alizecharge.fr', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
230,1218,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE914790181,2024-02-27,2024-02-27T11:11:36.071000+00:00
231,1219,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901810,2024-02-27,2024-02-27T11:11:36.071000+00:00
232,1220,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901811,2024-02-27,2024-02-27T11:11:36.071000+00:00
233,1221,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901812,2024-02-27,2024-02-27T11:11:36.071000+00:00
234,1222,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901813,2024-02-27,2024-02-27T11:11:36.071000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2292,80816,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550401,2024-02-27,2024-02-27T11:11:36.071000+00:00
2293,80817,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550402,2024-02-27,2024-02-27T11:11:36.071000+00:00
2294,80818,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550403,2024-02-27,2024-02-27T11:11:36.071000+00:00
2295,80819,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550404,2024-02-27,2024-02-27T11:11:36.071000+00:00


In [25]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRADPP91479018', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1218,1218,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE914790181,2024-02-27,2024-02-27T11:11:36.071000+00:00
1219,1219,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901810,2024-02-27,2024-02-27T11:11:36.071000+00:00
1220,1220,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901811,2024-02-27,2024-02-27T11:11:36.071000+00:00
1221,1221,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901812,2024-02-27,2024-02-27T11:11:36.071000+00:00
1222,1222,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901813,2024-02-27,2024-02-27T11:11:36.071000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,1295,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-P...,FRADPP91479018,ORY - Parking P3 SILO - Niveau 2,Voirie,16,Accès libre,Mo-Su 00:00-23:59,False,FRADPE914790188,2024-02-27,2024-02-27T11:11:36.071000+00:00
1296,1296,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-Poste,FRADPP91479018,ORY - Parking P3 SILO - Niveau 6,Voirie,18,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901880,2024-02-27,2024-02-27T11:11:36.071000+00:00
1297,1297,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-Poste,FRADPP91479018,ORY - Parking P3 SILO - Niveau 6,Voirie,18,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901881,2024-02-27,2024-02-27T11:11:36.071000+00:00
1298,1298,support@alizecharge.fr,ADP Groupe,"[2.362554, 48.728787]",2 Rue des Transporteurs 94390 Paray-Vieille-Poste,FRADPP91479018,ORY - Parking P3 SILO - Niveau 6,Voirie,18,Accès libre,Mo-Su 00:00-23:59,False,FRADPE9147901882,2024-02-27,2024-02-27T11:11:36.071000+00:00


In [26]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRTLSP31555040', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
80047,80816,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550401,2024-02-27,2024-02-27T11:11:36.071000+00:00
80048,80817,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550402,2024-02-27,2024-02-27T11:11:36.071000+00:00
80049,80818,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550403,2024-02-27,2024-02-27T11:11:36.071000+00:00
80050,80819,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550404,2024-02-27,2024-02-27T11:11:36.071000+00:00
80051,80820,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550405,2024-02-27,2024-02-27T11:11:36.071000+00:00


### Defauts operateur IZIVIA
Les défauts au nombre de 280 sont liés à une incohérence entre adresse et coordonnées (une même coordonnée a plusieurs adresses différentes). Ce défaut est lié soit à des erreurs de saisie, soit à des ajouts (ou remplacements) ultérieurs de stations à une même localisation.

Les stations Izivia en erreur ont un id_station_itinerance identique à l'id_pdc_itinerance, ce qui se traduit par des adresses multiples pour une même coordonnée.

Par exemple, sur un parking [-0.530598, 47.398424], on trouve 17 stations avec la même coordonnée, le même identifiant entre point de recharge et station et un nombre de PdC indiqué entre 1 et 3.

In [27]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'sav@izivia.com', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1555,45225,sav@izivia.com,EDF-EV100,"[5.266002, 45.802265]",400 AVENUE COTIER,FROTHPIZIG111101,CNPE BUGEY - CAMPUS UFPI - REMPLACEMENT,Voirie,1,Accès libre,24/7,false,FROTHEIZIG111101,2024-02-06,2024-02-07T15:13:25.625000+00:00
1556,45226,sav@izivia.com,EDF-EV100,"[5.266002, 45.802265]",400 AVENUE COTIER,FROTHPIZIG11111,CNPE BUGEY - CAMPUS UFPI - REMPLACEMENT,Voirie,1,Accès libre,24/7,false,FROTHEIZIG11111,2024-02-06,2024-02-07T15:13:25.625000+00:00
1557,45227,sav@izivia.com,EDF-EV100,"[5.266002, 45.802265]",400 AVENUE COTIER,FROTHPIZIG11221,CNPE BUGEY - CAMPUS UFPI - REMPLACEMENT,Voirie,1,Accès libre,24/7,false,FROTHEIZIG11221,2024-02-06,2024-02-07T15:13:25.625000+00:00
1558,45228,sav@izivia.com,EDF-EV100,"[5.266002, 45.802265]",400 AVENUE COTIER,FROTHPIZIG11331,CNPE BUGEY - CAMPUS UFPI - REMPLACEMENT,Voirie,1,Accès libre,24/7,false,FROTHEIZIG11331,2024-02-06,2024-02-07T15:13:25.625000+00:00
1559,45229,sav@izivia.com,EDF-EV100,"[5.266002, 45.802265]",400 AVENUE COTIER,FROTHPIZIG11441,CNPE BUGEY - CAMPUS UFPI - REMPLACEMENT,Voirie,1,Accès libre,24/7,false,FROTHEIZIG11441,2024-02-06,2024-02-07T15:13:25.625000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1784,46202,sav@izivia.com,EDF-EV100,"[1.57782, 47.71997]",LA CENTRALE NUCLEAIRECNPE SAINT-LAURENT DES EAUX,FROTHPIZIG47351,CNPE SAINT-LAURENT DES EAUX - E - RANGEE 6,Voirie,1,Accès libre,24/7,false,FROTHEIZIG47351,2024-02-06,2024-02-07T15:13:25.625000+00:00
1785,46203,sav@izivia.com,EDF-EV100,"[1.57782, 47.71997]",LA CENTRALE NUCLEAIRECNPE SAINT-LAURENT DES EAUX,FROTHPIZIG47361,CNPE SAINT-LAURENT DES EAUX - E - RANGEE 6,Voirie,1,Accès libre,24/7,false,FROTHEIZIG47361,2024-02-06,2024-02-07T15:13:25.625000+00:00
1786,46204,sav@izivia.com,EDF-EV100,"[1.57782, 47.71997]",LA CENTRALE NUCLEAIRECNPE SAINT-LAURENT DES EAUX,FROTHPIZIG47371,CNPE SAINT-LAURENT DES EAUX - E - RANGEE 6,Voirie,1,Accès libre,24/7,false,FROTHEIZIG47371,2024-02-06,2024-02-07T15:13:25.625000+00:00
1787,46205,sav@izivia.com,EDF-EV100,"[1.57782, 47.71997]",LA CENTRALE NUCLEAIRECNPE SAINT-LAURENT DES EAUX,FROTHPIZIG47381,CNPE SAINT-LAURENT DES EAUX - E - RANGEE 6,Voirie,1,Accès libre,24/7,false,FROTHEIZIG47381,2024-02-06,2024-02-07T15:13:25.625000+00:00


In [28]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-0.530598, 47.398424]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### Defauts operateur IONITY
Les défauts (480 points de recharge) sont liés à une incohérence entre adresse et coordonnées (une même coordonnée a plusieurs adresses différentes). Ce défaut est lié soit à des erreurs de saisie, soit à des ajouts (ou remplacements) ultérieurs de stations à une même localisation.

Les stations Ionity en erreur ont un id_station_itinerance identique à l'id_pdc_itinerance ainsi qu'un ancien pdc de regroupement par station avec un nom d'enseigne différent (IONITY GMBH), ce qui se traduit par des adresses multiples pour une même coordonnée.

Par exemple, sur l'aire de Mornas [4.732409, 44.194832], on trouve 26 stations avec la même coordonnée, le même identifiant entre point de recharge et station et un nombre de PdC indiqué de 23 pour chaque station.

In [29]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'info@ionity.eu', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
916,30391,info@ionity.eu,IONITY GMBH,"[0.366838, 46.701436]","Aire de Poitou-Charentes, A10, 79180 Vouillé",FRIONE124800,IONITY Poitou Charentes,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1248,2023-03-21,2024-01-19T07:47:22.735000+00:00
917,30392,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2024-01-19T07:47:22.735000+00:00
918,30393,info@ionity.eu,IONITY GMBH,"[1.578674, 48.467261]","Aire De Chartres Bois,A11 - 28300 Gasville-Oiséme",FRIONE126500,IONITY Chartres Bois Paris,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1265,2023-03-21,2024-01-19T07:47:22.735000+00:00
919,30394,info@ionity.eu,IONITY GMBH,"[3.918671, 49.242974]","Aire de Vrigny,A4-51390 Vrigny",FRIONE401700,IONITY Vrigny,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4017,2023-03-21,2024-01-19T07:47:22.735000+00:00
920,30397,info@ionity.eu,IONITY GMBH,"[1.416918, 46.313059]","Aire de Boismandé Ouest, A20 - 87160 Saint-Sul...",FRIONE402000,IONITY Boismandé Ouest,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4020,2023-03-21,2024-01-19T07:47:22.735000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,31455,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440353,Mornas Village,Station dédiée à la recharge rapide,23,Accès libre,24/7,false,FRIOYE440353,2024-02-27,2024-02-28T00:00:20.125000+00:00
1335,31456,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440371,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440371,2024-02-27,2024-02-28T00:00:20.125000+00:00
1336,31457,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440372,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440372,2024-02-27,2024-02-28T00:00:20.125000+00:00
1337,31458,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440373,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440373,2024-02-27,2024-02-28T00:00:20.125000+00:00


In [30]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[1.580008, 48.468782]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
917,30392,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2024-01-19T07:47:22.735000+00:00
964,30535,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126401,Chartres Gasville,Station dédiée à la recharge rapide,7,Accès libre,24/7,false,FRIOYE126401,2024-02-27,2024-02-28T00:00:20.125000+00:00
965,30536,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126402,Chartres Gasville,Station dédiée à la recharge rapide,7,Accès libre,24/7,false,FRIOYE126402,2024-02-27,2024-02-28T00:00:20.125000+00:00
966,30537,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126404,Chartres Gasville,Station dédiée à la recharge rapide,7,Accès libre,24/7,false,FRIOYE126404,2024-02-27,2024-02-28T00:00:20.125000+00:00
967,30538,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126405,Chartres Gasville,Station dédiée à la recharge rapide,7,Accès libre,24/7,false,FRIOYE126405,2024-02-27,2024-02-28T00:00:20.125000+00:00
968,30539,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126451,Chartres Gasville,Voirie,7,Accès libre,24/7,false,FRIOYE126451,2024-02-27,2024-02-28T00:00:20.125000+00:00
969,30540,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126452,Chartres Gasville,Station dédiée à la recharge rapide,7,Accès libre,24/7,false,FRIOYE126452,2024-02-27,2024-02-28T00:00:20.125000+00:00
970,30541,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126453,Chartres Gasville,Station dédiée à la recharge rapide,7,Accès libre,24/7,false,FRIOYE126453,2024-02-27,2024-02-28T00:00:20.125000+00:00


In [31]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[4.732409, 44.194832]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
962,30498,info@ionity.eu,IONITY GMBH,"[4.732409, 44.194832]","Aire de Mornas Village , A7, 84550 Mornas",FRIONE440300,IONITY Mornas Village,Station dédiée à la recharge rapide,16,Accès libre,24/7,FALSE,FRIONE4403,2023-03-21,2024-01-19T07:47:22.735000+00:00
1316,31437,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440301,Mornas Village,Station dédiée à la recharge rapide,23,Accès libre,24/7,false,FRIOYE440301,2024-02-27,2024-02-28T00:00:20.125000+00:00
1317,31438,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440302,Mornas Village,Station dédiée à la recharge rapide,23,Accès libre,24/7,false,FRIOYE440302,2024-02-27,2024-02-28T00:00:20.125000+00:00
1318,31439,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440303,Mornas Village,Station dédiée à la recharge rapide,23,Accès libre,24/7,false,FRIOYE440303,2024-02-27,2024-02-28T00:00:20.125000+00:00
1319,31440,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440304,Mornas Village,Station dédiée à la recharge rapide,23,Accès libre,24/7,false,FRIOYE440304,2024-02-27,2024-02-28T00:00:20.125000+00:00
1320,31441,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440305,Mornas Village,Station dédiée à la recharge rapide,23,Accès libre,24/7,false,FRIOYE440305,2024-02-27,2024-02-28T00:00:20.125000+00:00
1321,31442,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440306,Mornas Village,Station dédiée à la recharge rapide,23,Accès libre,24/7,false,FRIOYE440306,2024-02-27,2024-02-28T00:00:20.125000+00:00
1322,31443,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440307,Mornas Village,Station dédiée à la recharge rapide,23,Accès libre,24/7,false,FRIOYE440307,2024-02-27,2024-02-28T00:00:20.125000+00:00
1323,31444,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440308,Mornas Village,Station dédiée à la recharge rapide,23,Accès libre,24/7,false,FRIOYE440308,2024-02-27,2024-02-28T00:00:20.125000+00:00
1324,31445,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440309,Mornas Village,Station dédiée à la recharge rapide,23,Accès libre,24/7,false,FRIOYE440309,2024-02-27,2024-02-28T00:00:20.125000+00:00


In [32]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'info@ionity.eu', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
916,30391,info@ionity.eu,IONITY GMBH,"[0.366838, 46.701436]","Aire de Poitou-Charentes, A10, 79180 Vouillé",FRIONE124800,IONITY Poitou Charentes,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1248,2023-03-21,2024-01-19T07:47:22.735000+00:00
917,30392,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2024-01-19T07:47:22.735000+00:00
918,30393,info@ionity.eu,IONITY GMBH,"[1.578674, 48.467261]","Aire De Chartres Bois,A11 - 28300 Gasville-Oiséme",FRIONE126500,IONITY Chartres Bois Paris,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1265,2023-03-21,2024-01-19T07:47:22.735000+00:00
919,30394,info@ionity.eu,IONITY GMBH,"[3.918671, 49.242974]","Aire de Vrigny,A4-51390 Vrigny",FRIONE401700,IONITY Vrigny,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4017,2023-03-21,2024-01-19T07:47:22.735000+00:00
920,30397,info@ionity.eu,IONITY GMBH,"[1.416918, 46.313059]","Aire de Boismandé Ouest, A20 - 87160 Saint-Sul...",FRIONE402000,IONITY Boismandé Ouest,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4020,2023-03-21,2024-01-19T07:47:22.735000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,31455,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440353,Mornas Village,Station dédiée à la recharge rapide,23,Accès libre,24/7,false,FRIOYE440353,2024-02-27,2024-02-28T00:00:20.125000+00:00
1335,31456,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440371,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440371,2024-02-27,2024-02-28T00:00:20.125000+00:00
1336,31457,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440372,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440372,2024-02-27,2024-02-28T00:00:20.125000+00:00
1337,31458,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440373,Mornas Village,Voirie,23,Accès libre,24/7,false,FRIOYE440373,2024-02-27,2024-02-28T00:00:20.125000+00:00


In [33]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[0.366838, 46.701436]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
916,30391,info@ionity.eu,IONITY GMBH,"[0.366838, 46.701436]","Aire de Poitou-Charentes, A10, 79180 Vouillé",FRIONE124800,IONITY Poitou Charentes,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1248,2023-03-21,2024-01-19T07:47:22.735000+00:00
1136,30866,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409401,Poitiers Jaunay-Clan,Station dédiée à la recharge rapide,7,Accès libre,24/7,false,FRIOYE409401,2024-02-27,2024-02-28T00:00:20.125000+00:00
1137,30867,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409402,Poitiers Jaunay-Clan,Station dédiée à la recharge rapide,7,Accès libre,24/7,false,FRIOYE409402,2024-02-27,2024-02-28T00:00:20.125000+00:00
1138,30868,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409403,Poitiers Jaunay-Clan,Station dédiée à la recharge rapide,7,Accès libre,24/7,false,FRIOYE409403,2024-02-27,2024-02-28T00:00:20.125000+00:00
1139,30869,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409404,Poitiers Jaunay-Clan,Station dédiée à la recharge rapide,7,Accès libre,24/7,false,FRIOYE409404,2024-02-27,2024-02-28T00:00:20.125000+00:00
1140,30870,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409451,Poitiers Jaunay-Clan,Voirie,7,Accès libre,24/7,false,FRIOYE409451,2024-02-27,2024-02-28T00:00:20.125000+00:00
1141,30871,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409452,Poitiers Jaunay-Clan,Station dédiée à la recharge rapide,7,Accès libre,24/7,false,FRIOYE409452,2024-02-27,2024-02-28T00:00:20.125000+00:00
1142,30872,info@ionity.eu,Poitiers Jaunay-Clan,"[0.366838, 46.701436]","Autoroute A10, 86130 Jaunay-Clan",FRIOYE409453,Poitiers Jaunay-Clan,Station dédiée à la recharge rapide,7,Accès libre,24/7,false,FRIOYE409453,2024-02-27,2024-02-28T00:00:20.125000+00:00


In [34]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[2.523685, 48.9908]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### Cohérence implantation_station - id_station
- 107 pdc sont liés à une erreur de choix d'implantation (couplé à d'autres erreurs)

In [35]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
42,335,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300611,2022-11-03,2024-02-05T15:16:49.826000+00:00
43,336,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Voirie,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300621,2022-11-03,2024-02-05T15:16:49.826000+00:00
44,337,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300631,2022-11-03,2024-02-05T15:16:49.826000+00:00
45,338,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300641,2022-11-03,2024-02-05T15:16:49.826000+00:00
477,6831,support@alizecharge.fr,Métropole Rouen Normandie,"[0.872578, 49.479934]",Avenue du Président Coty 76480 DUCLAIR,FRC01P76222001,DUCLAIR - Avenue du Président Coty,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRC01E762220011,2024-02-27,2024-02-27T11:11:36.071000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2292,80816,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550401,2024-02-27,2024-02-27T11:11:36.071000+00:00
2293,80817,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550402,2024-02-27,2024-02-27T11:11:36.071000+00:00
2294,80818,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550403,2024-02-27,2024-02-27T11:11:36.071000+00:00
2295,80819,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550404,2024-02-27,2024-02-27T11:11:36.071000+00:00


### Cohérence nom_station - id_station
- 210 pdc sont associés à une station avec un nom non cohérent

In [36]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
47,343,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.764347, 43.958037]",AVENUE DU GENERAL DE GAULE - LES ANGLES,FR55CP30133,AVENUE DU GENERAL DE GAULE - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395803747643461,2022-11-03,2024-02-05T15:16:49.826000+00:00
48,344,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.764347, 43.958037]",AVENUE DU GENERAL DE GAULE - LES ANGLES,FR55CP30133,AVENUE DU GENERAL DE GAULE - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395803747643471,2022-11-03,2024-02-05T15:16:49.826000+00:00
49,345,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.752516, 43.962289]",PARKING PRIADES - LES ANGLES,FR55CP30133,PARKING PRIADES - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395842547684251,2022-11-03,2024-02-05T15:16:49.826000+00:00
50,346,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.752516, 43.962289]",PARKING PRIADES - LES ANGLES,FR55CP30133,PARKING PRIADES - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395842547684261,2022-11-03,2024-02-05T15:16:49.826000+00:00
51,347,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.768425, 43.958425]",PARKING ELEPHANT BLEU - LES ANGLES,FR55CP30133,PARKING ELEPHANT BLEU - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334396228947525161,2022-11-03,2024-02-05T15:16:49.826000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2292,80816,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550401,2024-02-27,2024-02-27T11:11:36.071000+00:00
2293,80817,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550402,2024-02-27,2024-02-27T11:11:36.071000+00:00
2294,80818,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550403,2024-02-27,2024-02-27T11:11:36.071000+00:00
2295,80819,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,False,FRTLSE315550404,2024-02-27,2024-02-27T11:11:36.071000+00:00


### Coherence adresse - coordonnees
- 1807 pdc ont une adresse non cohérente avec les coordonnées géographiques

In [37]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
18,133,exploitation@r3-charge.fr,R3,"[3.472424, 50.379911]","400 Rue Michel Chasles, 59494 Petite-Forêt",FR3R3P89616641,R3 - Petite Forêt,Parking privé à usage public,5,Accès libre,Mo-Su 07:00-22:00,false,FR3R3E10000849681,2023-12-22,2024-01-29T07:15:04.845000+00:00
19,134,exploitation@r3-charge.fr,R3,"[3.472424, 50.379911]","401 Rue Michel Chasles, 59494 Petite-Forêt",FR3R3P89616641,R3 - Petite Forêt,Parking privé à usage public,5,Accès libre,Mo-Su 07:00-22:01,false,FR3R3E10000849682,2023-12-22,2024-01-29T07:15:04.845000+00:00
20,135,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",708 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849691,2023-12-22,2024-01-29T07:15:04.845000+00:00
21,136,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",709 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849692,2023-12-22,2024-01-29T07:15:04.845000+00:00
22,153,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",713 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849851,2023-12-22,2024-01-29T07:15:04.845000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2823,87809,advenir@zeborne.com,GARAGE NELLO CHELLI - DIJON,"[5.065799, 47.354481]",23 RUE DES ARDENNES 21000 Dijon,FRZTLE22AC55087,Toyota - Dijon - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 07:45-12:00,Mo-Fr 13:45-19:00,Sat 09:00-...",false,FRZTLE22AC55087,2021-02-19,2024-01-19T07:46:24.141000+00:00
2824,87818,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2024-01-19T07:46:23.908000+00:00
2825,87819,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2024-01-19T07:46:23.908000+00:00
2826,87820,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2024-01-19T07:46:23.908000+00:00


In [38]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[5.2592028, 46.1746523]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2824,87818,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2024-01-19T07:46:23.908000+00:00
2825,87819,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2024-01-19T07:46:23.908000+00:00
2826,87820,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2024-01-19T07:46:23.908000+00:00
2827,87821,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57335,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57335,2021-04-08,2024-01-19T07:46:23.908000+00:00


In [39]:
itinerance_4.loc[itinerance_4.adresse_station == 'RN 75 ZI DE NOIREFONTAINE 01250 Montagnat', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2824,87818,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2024-01-19T07:46:23.908000+00:00
2826,87820,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2024-01-19T07:46:23.908000+00:00


In [40]:
itinerance_4.loc[itinerance_4.adresse_station == ' RN 75 ZI DE NOIREFONTAINE 01250 Montagnat', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2825,87819,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2024-01-19T07:46:23.908000+00:00
2827,87821,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57335,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57335,2021-04-08,2024-01-19T07:46:23.908000+00:00


- exemple : Stations Ouest charge (double saisie par deux opérateurs)

In [41]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-3.577855, 48.771084]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### Cohérence station - coordonnées
- 323 pdc sont associés à des stations avec plusieurs coordonnées (dont 127 Freshmile)

In [42]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations][300:315]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1811,56944,hello@powerdot.fr,Power Dot France,"[-0.59952, 44.73295]",2 Av. de Bordeaux,FRPD1PSYULGN,SUPER U LEOGNAN,Parking privé à usage public,5,Accès libre,24/7,False,FRPD1ESYULGNALF22011,2024-01-09,2024-01-19T07:47:08.664000+00:00
1812,56945,hello@powerdot.fr,Power Dot France,"[-0.59952, 44.73295]",2 Av. de Bordeaux,FRPD1PSYULGN,SUPER U LEOGNAN,Parking privé à usage public,5,Accès libre,24/7,False,FRPD1ESYULGNBB200011,2024-01-09,2024-01-19T07:47:08.664000+00:00
1813,56946,hello@powerdot.fr,Power Dot France,"[-0.59952, 44.73295]",2 Av. de Bordeaux,FRPD1PSYULGN,SUPER U LEOGNAN,Parking privé à usage public,5,Accès libre,24/7,False,FRPD1ESYULGNBB200012,2024-01-09,2024-01-19T07:47:08.664000+00:00
1814,56947,hello@powerdot.fr,Power Dot France,"[-0.59952, 44.73295]",2 Av. de Bordeaux,FRPD1PSYULGN,SUPER U LEOGNAN,Parking privé à usage public,5,Accès libre,24/7,False,FRPD1ESYULGNBB200013,2024-01-09,2024-01-19T07:47:08.664000+00:00
1815,56948,hello@powerdot.fr,Power Dot France,"[-0.59952, 44.73295]",2 Av. de Bordeaux,FRPD1PSYULGN,SUPER U LEOGNAN,Parking privé à usage public,5,Accès libre,24/7,False,FRPD1ESYULGNBB200014,2024-01-09,2024-01-19T07:47:08.664000+00:00
1816,56949,hello@powerdot.fr,Power Dot France,"[1.7734888, 45.2456586]",1 Av. de Coulaud,FRPD1PSYULGN,Super U Laguenne,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1ESYULGNEKO60011,2024-01-09,2024-01-19T07:47:08.664000+00:00
1817,56950,hello@powerdot.fr,Power Dot France,"[1.7734888, 45.2456586]",1 Av. de Coulaud,FRPD1PSYULGN,Super U Laguenne,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1ESYULGNEKO60012,2024-01-09,2024-01-19T07:47:08.664000+00:00
1818,56951,hello@powerdot.fr,Power Dot France,"[1.7734888, 45.2456586]",1 Av. de Coulaud,FRPD1PSYULGN,Super U Laguenne,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1ESYULGNEKO60013,2024-01-09,2024-01-19T07:47:08.664000+00:00
1819,56952,hello@powerdot.fr,Power Dot France,"[1.7734888, 45.2456586]",1 Av. de Coulaud,FRPD1PSYULGN,Super U Laguenne,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1ESYULGNEKO60021,2024-01-09,2024-01-19T07:47:08.664000+00:00
1820,56953,hello@powerdot.fr,Power Dot France,"[1.7734888, 45.2456586]",1 Av. de Coulaud,FRPD1PSYULGN,Super U Laguenne,Parking privé à usage public,8,Accès libre,24/7,False,FRPD1ESYULGNEKO60022,2024-01-09,2024-01-19T07:47:08.664000+00:00


In [43]:
itinerance_5.loc[itinerance_5.id_station_itinerance	 == 'FRLE2PYSCJHWOOBD', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


In [44]:
itinerance_5.loc[itinerance_5.id_station_itinerance	 == 'FRFR1PEHNKRRPQNR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
526,22192,web@freshmile.com,Freshmile,"[1.391725, 43.566937]",90 avenue d'Embrun 05000 Gap,FRFR1PEHNKRRPQNR,"Gap, 90 avenue d'Embrun - 3273",Voirie,6,Accès libre,24/7,false,FRFR1EJVVE1,2022-11-09,2024-02-14T06:40:03.247000+00:00
527,22193,web@freshmile.com,Freshmile,"[0.66337, 43.85402]",90 avenue d'Embrun 05000 Gap,FRFR1PEHNKRRPQNR,"Gap, 90 avenue d'Embrun - 3273",Voirie,6,Accès libre,24/7,false,FRFR1EJVVE2,2022-11-09,2024-02-14T06:40:03.247000+00:00
541,24532,web@freshmile.com,Freshmile,"[1.523894, 44.147449]",90 avenue d'Embrun 05000 Gap,FRFR1PEHNKRRPQNR,"Gap, 90 avenue d'Embrun - 3273",Voirie,6,Accès libre,24/7,false,FRFR1ERHZF1,2022-11-09,2024-02-14T06:40:03.247000+00:00
542,24533,web@freshmile.com,Freshmile,"[4.636316, 44.26687]",90 avenue d'Embrun 05000 Gap,FRFR1PEHNKRRPQNR,"Gap, 90 avenue d'Embrun - 3273",Voirie,6,Accès libre,24/7,false,FRFR1ERHZF2,2022-11-09,2024-02-14T06:40:03.247000+00:00


- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [45]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,18,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2024-01-19T07:46:04.014000+00:00
1,19,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2024-01-19T07:46:04.014000+00:00
2,20,acelec@acelec-france.com,Camaïeu France,"[3.207124,50.684984]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EBEQA5957,2022-07-21,2024-01-19T07:46:04.014000+00:00
3,21,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2024-01-19T07:46:04.014000+00:00
5,23,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EK0BJ522,2022-07-21,2024-01-19T07:46:04.014000+00:00
6,24,acelec@acelec-france.com,Camaïeu France,"[3.207047,50.685013]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EMZLV5283,2022-07-21,2024-01-19T07:46:04.014000+00:00
7,25,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EO0AV51091,2022-07-21,2024-01-19T07:46:04.014000+00:00
8,26,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP7QU5233,2022-07-21,2024-01-19T07:46:04.014000+00:00
9,27,acelec@acelec-france.com,Camaïeu France,"[3.206962,50.685049]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP9LD51287,2022-07-21,2024-01-19T07:46:04.014000+00:00
10,28,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EQ1GI51176,2022-07-21,2024-01-19T07:46:04.014000+00:00


- exemple : une station avec plusieurs coordonnées -> regroupement de stations ?

In [46]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR55CPBP514', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
52,354,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6A61,2022-11-03,2024-02-05T15:16:49.826000+00:00
53,355,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B62,2022-11-03,2024-02-05T15:16:49.826000+00:00
54,356,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B63,2022-11-03,2024-02-05T15:16:49.826000+00:00
55,357,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A1,2022-11-03,2024-02-05T15:16:49.826000+00:00
56,358,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A2,2022-11-03,2024-02-05T15:16:49.826000+00:00
57,359,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A3,2022-11-03,2024-02-05T15:16:49.826000+00:00


- exemple : plusieurs stations d'opérateurs différents avec les mêmes coordonnées -> suppression non effectuée lors d'un changement d'opérateur ?

In [47]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[0.08745032, 48.40887156]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1401,40101,web@freshmile.com,Freshmile,"[0.08745032, 48.40887156]",Route Du Mans 72610 Arçonnay,FRLE2PR7NKMNWNJE,"E.Leclerc, Alençon Arconnay",Voirie,24,Accès libre,24/7,false,FRLE2EWFVS1,2024-01-31,2024-02-14T06:40:03.247000+00:00


In [48]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-4.374469, 48.471878]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


- exemple : quatre stations avec une localisation identique mais deux adresses différentes + id identique entre station et pdc -> erreur d'adresse et erreur d'id_station ?

In [49]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-0.366184, 43.332508]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### Incoherence nbre_pdc
- 583 pdc ont un champ 'nbre_pdc' mal documenté

In [50]:
itinerance_4.loc[~itinerance_4['nbre_pdc - id_station_itinerance'], relations][200:215]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
620,18166,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.0697451419812665, 50.66179854122704]",Rue Gustave Scrive 59110 La Madeleine,FRETIP59368A,SUPER U La Madeleine,Parking privé à usage public,11,Accès libre,24/7,false,FRETIE59368A23,2023-12-22,2024-01-19T07:48:33.087000+00:00
621,18167,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.0697451419812665, 50.66179854122704]",Rue Gustave Scrive 59110 La Madeleine,FRETIP59368A,SUPER U La Madeleine,Parking privé à usage public,11,Accès libre,24/7,false,FRETIE59368A24,2023-12-22,2024-01-19T07:48:33.087000+00:00
622,18168,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.0697451419812665, 50.66179854122704]",Rue Gustave Scrive 59110 La Madeleine,FRETIP59368A,SUPER U La Madeleine,Parking privé à usage public,11,Accès libre,24/7,false,FRETIE59368A25,2023-12-22,2024-01-19T07:48:33.087000+00:00
623,18169,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.0697451419812665, 50.66179854122704]",Rue Gustave Scrive 59110 La Madeleine,FRETIP59368A,SUPER U La Madeleine,Parking privé à usage public,11,Accès libre,24/7,false,FRETIE59368A26,2023-12-22,2024-01-19T07:48:33.087000+00:00
624,18170,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.0697451419812665, 50.66179854122704]",Rue Gustave Scrive 59110 La Madeleine,FRETIP59368A,SUPER U La Madeleine,Parking privé à usage public,11,Accès libre,24/7,false,FRETIE59368A27,2023-12-22,2024-01-19T07:48:33.087000+00:00
625,18171,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.0697451419812665, 50.66179854122704]",Rue Gustave Scrive 59110 La Madeleine,FRETIP59368A,SUPER U La Madeleine,Parking privé à usage public,11,Accès libre,24/7,false,FRETIE59368A28,2023-12-22,2024-01-19T07:48:33.087000+00:00
626,18172,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.0697451419812665, 50.66179854122704]",Rue Gustave Scrive 59110 La Madeleine,FRETIP59368A,SUPER U La Madeleine,Parking privé à usage public,11,Accès libre,24/7,false,FRETIE59368A29,2023-12-22,2024-01-19T07:48:33.087000+00:00
627,18173,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.0697451419812665, 50.66179854122704]",Rue Gustave Scrive 59110 La Madeleine,FRETIP59368A,SUPER U La Madeleine,Parking privé à usage public,4,Accès libre,24/7,false,FRETIE59368A31,2023-12-22,2024-01-19T07:48:33.087000+00:00
628,18174,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.0697451419812665, 50.66179854122704]",Rue Gustave Scrive 59110 La Madeleine,FRETIP59368A,SUPER U La Madeleine,Parking privé à usage public,4,Accès libre,24/7,false,FRETIE59368A32,2023-12-22,2024-01-19T07:48:33.087000+00:00
629,18175,contact@e-totem.fr,Réseau e-Totem Infrastructures,"[3.0697451419812665, 50.66179854122704]",Rue Gustave Scrive 59110 La Madeleine,FRETIP59368A,SUPER U La Madeleine,Parking privé à usage public,4,Accès libre,24/7,false,FRETIE59368A33,2023-12-22,2024-01-19T07:48:33.087000+00:00


### coherence station - enseigne
- 5 stations ont plusieurs noms d'enseigne

In [51]:
itinerance_4.loc[~itinerance_4['nom_enseigne - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
650,20849,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2024-01-19T07:50:45.527000+00:00
651,20850,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2024-01-19T07:50:45.527000+00:00
652,20851,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS3,2022-12-09,2024-01-19T07:50:45.527000+00:00
1415,41933,contact@rirodo.fr,LAST MILES SOLUTIONS,"[5.39, 43.27]",20 Rue Negresko,FRRIRE2804856,PARKING NEGRESKO,Parking privé à usage public,2,Accès libre,24/7,false,FRLMSE1897228,2023-09-06,2024-01-19T07:49:16.739000+00:00
1832,57484,contact@rirodo.fr,MONTA,"[5.39, 43.27]",20 Rue NEGRESKO,FRRIRE2804856,PARKING NEGRESKO,Parking privé à usage public,2,Accès libre,24/7,false,FRRIRE2804856,2023-09-06,2024-01-19T07:49:16.739000+00:00


In [52]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRFR1EAYRU', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
20116,20130,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel jean colonna d'ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU1,2022-12-25,2024-01-19T07:49:57.766000+00:00
20118,20132,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU2,2022-08-25,2024-01-19T07:49:57.766000+00:00
20120,20134,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU3,2022-08-25,2024-01-19T07:49:57.766000+00:00
20835,20849,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2024-01-19T07:50:45.527000+00:00
20836,20850,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2024-01-19T07:50:45.527000+00:00
20837,20851,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS3,2022-12-09,2024-01-19T07:50:45.527000+00:00


In [53]:
itinerance_init.loc[itinerance_init.nom_enseigne == 'RME Loos (FR-FR1)', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
20116,20130,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel jean colonna d'ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU1,2022-12-25,2024-01-19T07:49:57.766000+00:00
20118,20132,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU2,2022-08-25,2024-01-19T07:49:57.766000+00:00
20835,20849,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2024-01-19T07:50:45.527000+00:00
20836,20850,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2024-01-19T07:50:45.527000+00:00
23796,23810,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.009613, 50.617509]",12 Rue Salengro LOOS,FRFR1EPJXS,"Loos, Parking CIL",Parking public,3,Accès libre,24/7,FALSE,FRFR1EPJXS1,2022-10-04,2024-01-19T07:49:31.469000+00:00
23798,23812,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.009613, 50.617509]",12 Rue Salengro LOOS,FRFR1EPJXS,"Loos, Parking CIL",Parking public,3,Accès libre,24/7,FALSE,FRFR1EPJXS2,2022-10-04,2024-01-19T07:49:31.469000+00:00


### cohérence station - horaires
- exemple de station avec plusieurs types d'horaire (15)

In [54]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS23D2302001', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### cohérence station - deux-roues
- exemple de station avec plusieurs indications d'accessibilité deux-roues (583)

In [55]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRV75PPX1716', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
84187,84956,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,TRUE,FRV75EPX17161,2023-07-06,2024-01-19T07:47:27.885000+00:00
84188,84957,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,TRUE,FRV75EPX17162,2023-07-06,2024-01-19T07:47:27.885000+00:00
84189,84958,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17163,2023-07-06,2024-01-19T07:47:27.885000+00:00
84190,84959,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17164,2023-07-06,2024-01-19T07:47:27.885000+00:00
84191,84960,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17165,2023-07-06,2024-01-19T07:47:27.885000+00:00
84192,84961,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17166,2023-07-06,2024-01-19T07:47:27.885000+00:00
84193,84962,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17167,2023-07-06,2024-01-19T07:47:27.885000+00:00
